In [11]:
import pandas as pd
import numpy as np
import glob
import os

In [12]:
#path
path_func = '/media/data/Dunedin_Study_Data_Narun_P_Jan2022/DBIS_NarunPat_retest_unstandardisedIQ/retests/fMRI_Regions.N379/'
path_str = '/media/data/Dunedin_Study_Data_Narun_P_Jan2022/DBIS_NarunPat_retest_unstandardisedIQ/retests/'
path_mov = '/media/data/Dunedin_Study_Data_Narun_P_Jan2022/DBIS_NarunPat_retest_unstandardisedIQ/retests/QC/MotionFiles/'

In [13]:
#labels
labels379 = pd.read_csv(path_func+'FC_labels.csv', header=None).values.ravel()
#display(labels379)

In [14]:
#load data
folders = [i.split('/')[-1] for i in sorted(glob.glob(path_func+'FC_*_matrices'))[2:]]
print(folders)

['FC_facename_matrices', 'FC_faces_matrices', 'FC_mid_matrices', 'FC_stroop_matrices']


In [15]:
#load data and calc cor matx, flatten them
folders = [i.split('/')[-1] for i in sorted(glob.glob(path_func+'FC_*_matrices'))[2:]]

dct_task_fc={}

for folder in folders:
    
    subjects = [i[:-4] for i in sorted(os.listdir(path_func+folder+'/'))]

    dct_group_fc={}

    for subject in subjects:
        df = pd.read_csv(path_func+folder+'/'+subject+'.csv', header=None)
        df.index, df.columns = labels379, labels379
        #r-2-z
        df = np.arctanh(df)
        #convert to string
        dfg = df.where(np.triu(np.ones(df.shape), k=1).astype(bool)).stack().reset_index()
        indx = [i+'_&_'+j for i,j in zip(dfg['level_0'], dfg['level_1'])]
        fc_string = pd.Series(np.array(dfg[0]),index=indx)
        #write a row to dict
        dct_group_fc[subject] = fc_string

    dct_task_fc[folder.split('_')[1]] = pd.DataFrame(dct_group_fc).T



In [16]:
#check for nan
for key in dct_task_fc.keys():
    print(key)
    print(dct_task_fc[key].isna().sum()[dct_task_fc[key].isna().sum()>0])

facename
Series([], dtype: int64)
faces
Series([], dtype: int64)
mid
Series([], dtype: int64)
stroop
Series([], dtype: int64)


In [ ]:
#sort indexes
for key in dct_task_fc.keys():
    #cut  index
    dct_task_fc[key].index = [''.join([i[:4], i[5:]]) for i in dct_task_fc[key].index]
    #sort index
    dct_task_fc[key] =  dct_task_fc[key].sort_index()
    
    print(key)
    display(dct_task_fc[key])

In [18]:
#save
for key in dct_task_fc.keys():
    dct_task_fc[key].to_csv('/media/data/Dunedin_Study_Data_Narun_P_Jan2022/MLTabs_retest/'+key+'_FC.csv')